<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/start_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.5 MB/s eta 0:00:00


In [5]:
from web3 import Web3
from eth_account import Account
import requests
import json
import time
import decimal
import numpy as np
import math

In [6]:
from web3 import exceptions
from eth_abi import encode
from typing import List, Dict, Any

In [4]:
ETHERSCAN_BASE = "https://api.etherscan.io/v2/api"  # مطابق نمونه شما (v2)
API_KEY = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
CHAIN_ID = 11155111  # Sepolia

In [7]:
# ---------- تنظیمات ----------
PAGE_SIZE = 500        # تعداد تراکنش در هر صفحه (max پیشنهاد شده را تست کن)
MAX_RETRIES = 4
INITIAL_BACKOFF = 0.3  # ثانیه
SLEEP_BETWEEN_PAGES = 0.15  # برای محافظت در برابر rate limit
# -----------------------------

def request_with_retry(params: Dict[str, Any], max_retries=MAX_RETRIES, backoff=INITIAL_BACKOFF) -> Dict[str, Any]:

    params = dict(params)
    params["apikey"] = API_KEY
    params["chainid"] = CHAIN_ID

    attempt = 0
    while True:
        attempt += 1
        try:
            resp = requests.get(ETHERSCAN_BASE, params=params, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            # ساختار v2 معمولاً {'status': '1'|'0', 'message': 'OK'|'No transactions', 'result': [...]}
            return data
        except Exception as e:
            if attempt >= max_retries:
                raise
            sleep_time = backoff * (2 ** (attempt - 1))
            print(f"[retry] attempt {attempt} failed: {e}. sleeping {sleep_time:.2f}s")
            time.sleep(sleep_time)

def get_tx_list_page(address: str, page: int = 1, offset: int = PAGE_SIZE, startblock: int = 0, endblock: int = 99999999, sort: str = "asc") -> List[Dict[str, Any]]:
    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "startblock": startblock,
        "endblock": endblock,
        "page": page,
        "offset": offset,
        "sort": sort
    }
    data = request_with_retry(params)
    if not isinstance(data, dict):
        raise Exception("Unexpected response type: " + str(type(data)))
    result = data.get("result")
    if result is None:
        raise Exception(f"Unexpected API response: {data}")
    if isinstance(result, str) and (result == "[]" or result == ""):
        return []
    return result if isinstance(result, list) else []

def fetch_all_txs(address: str, startblock: int = 0, endblock: int = 99999999, page_size: int = PAGE_SIZE, sort: str = "asc") -> List[Dict[str, Any]]:
    all_txs: List[Dict[str, Any]] = []
    page = 1
    while True:
        print(f"Fetching page {page} ...")
        page_items = get_tx_list_page(address, page=page, offset=page_size, startblock=startblock, endblock=endblock, sort=sort)
        if not page_items:
            print("No more transactions (empty page).")
            break
        all_txs.extend(page_items)
        print(f"  got {len(page_items)} txs; total so far: {len(all_txs)}")
        if len(page_items) < page_size:
            break
        page += 1
        time.sleep(SLEEP_BETWEEN_PAGES)
    return all_txs


WALLET = "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822"
START_BLOCK = 0
END_BLOCK = 99999999

print("Starting fetch txs for:", WALLET)
txs = fetch_all_txs(WALLET, startblock=START_BLOCK, endblock=END_BLOCK, page_size=200)
print("Total transactions fetched:", len(txs))

out_file = f"{WALLET}_txs.json"
with open(out_file, "w", encoding="utf-8") as f:
  json.dump(txs, f, ensure_ascii=False, indent=2)
print("Saved to", out_file)


Starting fetch txs for: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822
Fetching page 1 ...
  got 120 txs; total so far: 120
Total transactions fetched: 120
Saved to 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822_txs.json


In [8]:
txs[0]

{'blockNumber': '9733717',
 'timeStamp': '1764447684',
 'hash': '0x6a1aab2cfb3af814cc2c0b2f95db578b87a50db520b62944390627a1c3a1ccbc',
 'nonce': '2363432',
 'blockHash': '0xd6345ed65592801514047c3f9a8387668665edbbf0c62dc46901d70cdbd828d6',
 'transactionIndex': '0',
 'from': '0xc6392ad8a14794ea57d237d12017e7295bea2363',
 'to': '0x7868edefc36a74eba220c75acfb05c59a4e55822',
 'value': '5000000000000000',
 'gas': '21000',
 'gasPrice': '30000000000',
 'isError': '0',
 'txreceipt_status': '1',
 'input': '0x',
 'contractAddress': '',
 'cumulativeGasUsed': '21000',
 'gasUsed': '21000',
 'confirmations': '99966',
 'methodId': '0x',
 'functionName': ''}

In [9]:
def find_key(data, target_key):
    results = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                results.append(value)
            else:
                results.extend(find_key(value, target_key))
    elif isinstance(data, list):
        for item in data:
            results.extend(find_key(item, target_key))
    return results

In [10]:
def find_key_path(data, target_key):
    """
    جستجو در دیکشنری تو در تو برای یک کلید مشخص و برگرداندن
    مسیر root-to-leaf به صورت دیکشنری نگهدارنده ساختار.
    """
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                return {key: value}
            elif isinstance(value, (dict, list)):
                found = find_key_path(value, target_key)
                if found:
                    return {key: found}  # مسیر تا کلید هدف
    elif isinstance(data, list):
        for item in data:
            found = find_key_path(item, target_key)
            if found:
                return found
    return None

In [11]:
txs_hash=find_key(txs,'hash')

In [12]:
def get_receipt(tx_hash, api_key, chain_id):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "module": "proxy",
        "action": "eth_getTransactionReceipt",
        "txhash": tx_hash,
        "apikey": api_key,
        "chainid": chain_id
    }

    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()
    return data


receipt = get_receipt(txs_hash[2], API_KEY, CHAIN_ID)

print(json.dumps(receipt, indent=2))


{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "blockHash": "0xe07a53a09606e20a411ab9e795301ce62cdc872ef56f6bbca3bd2ce2496d6e9f",
    "blockNumber": "0x94a077",
    "contractAddress": null,
    "cumulativeGasUsed": "0x204fed",
    "effectiveGasPrice": "0x3b9aca08",
    "from": "0x7868edefc36a74eba220c75acfb05c59a4e55822",
    "gasUsed": "0xb052",
    "logs": [
      {
        "address": "0xfff9976782d46cc05630d1f6ebab18b2324d6b14",
        "topics": [
          "0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c",
          "0x0000000000000000000000007868edefc36a74eba220c75acfb05c59a4e55822"
        ],
        "data": "0x00000000000000000000000000000000000000000000000000038d7ea4c68000",
        "blockNumber": "0x94a077",
        "transactionHash": "0xf952d08bafc9b2c7fa777a66f570d7809fa4f2758a5df3bca69deb5ae941844d",
        "transactionIndex": "0x16",
        "blockHash": "0xe07a53a09606e20a411ab9e795301ce62cdc872ef56f6bbca3bd2ce2496d6e9f",
        "blockTimestamp":

In [13]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
my_wallet_address = account.address

print("Connected:", w3.is_connected())
print("Your address:", my_wallet_address)

Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [14]:
def fetch_implementation_address(contract_address: str, api_key: str, inp_chain_id: int):
    try:
        contract_address = Web3.to_checksum_address(contract_address)
    except ValueError:
        return None

    url = "https://api.etherscan.io/v2/api"
    params = {
        "module": "contract",
        "action": "getsourcecode", # اکشن تغییر کرد
        "address": contract_address,
        "apikey": api_key,
        "chainid": inp_chain_id
    }

    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        if data.get("status") == "1" and data["result"]:
            result = data["result"][0]
            implementation_address = result.get('Implementation')

            if implementation_address and implementation_address != '0x0000000000000000000000000000000000000000':
                return Web3.to_checksum_address(implementation_address)

    except Exception as e:
        print(f"Error checking implementation for {contract_address}: {e}")

    return None

In [15]:
def fetch_abi_retry(contract_address: str, api_key: str, inp_chain_id: int, retries=3):
    try:
        initial_address = Web3.to_checksum_address(contract_address)
    except ValueError:
        return None

    implementation_address = fetch_implementation_address(initial_address, api_key, inp_chain_id)

    addresses_to_check = []
    if implementation_address:
        addresses_to_check.append(implementation_address)
    addresses_to_check.append(initial_address)

    for current_addr in addresses_to_check:
        url = "https://api.etherscan.io/v2/api"
        params = {
            "module": "contract",
            "action": "getabi",
            "address": current_addr,
            "apikey": api_key,
            "chainid": inp_chain_id
        }

        for attempt in range(retries):
            try:
                resp = requests.get(url, params=params, timeout=10)
                resp.raise_for_status()
                data = resp.json()

                if data.get("status") == "1":
                    print(f"Successfully fetched ABI from: {current_addr}")
                    return json.loads(data["result"])


            except requests.exceptions.RequestException as e:
                time.sleep(1)

        time.sleep(0.5)

    return None

In [16]:
receipt = get_receipt(txs_hash[2], API_KEY, CHAIN_ID)
receipt

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'blockHash': '0xe07a53a09606e20a411ab9e795301ce62cdc872ef56f6bbca3bd2ce2496d6e9f',
  'blockNumber': '0x94a077',
  'contractAddress': None,
  'cumulativeGasUsed': '0x204fed',
  'effectiveGasPrice': '0x3b9aca08',
  'from': '0x7868edefc36a74eba220c75acfb05c59a4e55822',
  'gasUsed': '0xb052',
  'logs': [{'address': '0xfff9976782d46cc05630d1f6ebab18b2324d6b14',
    'topics': ['0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c',
     '0x0000000000000000000000007868edefc36a74eba220c75acfb05c59a4e55822'],
    'data': '0x00000000000000000000000000000000000000000000000000038d7ea4c68000',
    'blockNumber': '0x94a077',
    'transactionHash': '0xf952d08bafc9b2c7fa777a66f570d7809fa4f2758a5df3bca69deb5ae941844d',
    'transactionIndex': '0x16',
    'blockHash': '0xe07a53a09606e20a411ab9e795301ce62cdc872ef56f6bbca3bd2ce2496d6e9f',
    'blockTimestamp': '0x692c8fd4',
    'logIndex': '0x22',
    'removed': False}],
  'logsBloom': '0x000000000040

In [17]:
logs=find_key(receipt,'logs')[0][0]
logs

{'address': '0xfff9976782d46cc05630d1f6ebab18b2324d6b14',
 'topics': ['0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c',
  '0x0000000000000000000000007868edefc36a74eba220c75acfb05c59a4e55822'],
 'data': '0x00000000000000000000000000000000000000000000000000038d7ea4c68000',
 'blockNumber': '0x94a077',
 'transactionHash': '0xf952d08bafc9b2c7fa777a66f570d7809fa4f2758a5df3bca69deb5ae941844d',
 'transactionIndex': '0x16',
 'blockHash': '0xe07a53a09606e20a411ab9e795301ce62cdc872ef56f6bbca3bd2ce2496d6e9f',
 'blockTimestamp': '0x692c8fd4',
 'logIndex': '0x22',
 'removed': False}

In [18]:
find_key_path(logs,'address')

{'address': '0xfff9976782d46cc05630d1f6ebab18b2324d6b14'}

In [19]:
find_key_path(logs,'topics')

{'topics': ['0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c',
  '0x0000000000000000000000007868edefc36a74eba220c75acfb05c59a4e55822']}

In [20]:
p={
    'module':'proxy',
    'action':'eth_getTransactionByHash',
    'txhash':txs_hash[2],
    'apikey':API_KEY,
    'chainid':CHAIN_ID
}
requests.get('https://api.etherscan.io/v2/api',params=p).json()

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'blockHash': '0xe07a53a09606e20a411ab9e795301ce62cdc872ef56f6bbca3bd2ce2496d6e9f',
  'blockNumber': '0x94a077',
  'from': '0x7868edefc36a74eba220c75acfb05c59a4e55822',
  'gas': '0x52230',
  'gasPrice': '0x3b9aca08',
  'maxFeePerGas': '0x3b9aca08',
  'maxPriorityFeePerGas': '0x3b9aca00',
  'hash': '0xf952d08bafc9b2c7fa777a66f570d7809fa4f2758a5df3bca69deb5ae941844d',
  'input': '0xd0e30db0756e697800000000000c',
  'nonce': '0x0',
  'to': '0xfff9976782d46cc05630d1f6ebab18b2324d6b14',
  'transactionIndex': '0x16',
  'value': '0x38d7ea4c68000',
  'type': '0x2',
  'accessList': [],
  'chainId': '0xaa36a7',
  'v': '0x0',
  'r': '0xa50f4525a8810badd8a55175a1141cd10c8bfecc65055990cd3a21a2b40cb8df',
  's': '0x7d6f0ccaff94442a72dd257b5580c53c2805efaed32951311182e267a71cff26',
  'yParity': '0x0'}}

In [21]:
from web3.datastructures import AttributeDict
from hexbytes import HexBytes

def get_contract_abi_safe(address, api_key, chain_id):
    if not address:
        return None
    try:
        return fetch_abi_retry(address, api_key, chain_id)
    except Exception as e:
        print(f"Failed to fetch ABI for {address}: {e}")
        return None

def decode_log_fixed(w3, log, api_key, chain_id):

    contract_address = log['address']

    # دریافت ABI
    abi = get_contract_abi_safe(contract_address, api_key, chain_id)
    if not abi:
        return {"error": "no_abi_found", "address": contract_address}

    contract = w3.eth.contract(address=contract_address, abi=abi)

    #
    log_topic0 = log['topics'][0]
    if hasattr(log_topic0, 'hex'):
        log_topic0 = log_topic0.hex()

    #
    for event_abi in abi:
        if event_abi.get("type") == "event":

            inputs = ",".join([param["type"] for param in event_abi["inputs"]])
            event_signature = f"{event_abi['name']}({inputs})"
            event_hash = w3.keccak(text=event_signature).hex()

            #
            if event_hash.lower() == log_topic0.lower():
                try:
                    #
                    #
                    #
                    decoded_event = contract.events[event_abi['name']]().process_log(log)
                    return {
                        "event_name": decoded_event.event,
                        "args": dict(decoded_event.args),
                        "address": contract_address
                    }
                except Exception as e:
                    print(f"Error processing matching event {event_abi['name']}: {e}")
                    return {"error": "process_log_failed"}

    return {"error": "event_not_found_in_abi"}

# ==========================================
#
# ==========================================

#
target_tx_hash = txs_hash[2]
print(f"Decoding logs for tx: {target_tx_hash}")

#
#
try:
    receipt = w3.eth.get_transaction_receipt(target_tx_hash)

    print(f"Transaction status: {receipt['status']}")

    all_decoded_logs = []
    for log in receipt['logs']:
        decoded = decode_log_fixed(w3, log, API_KEY, CHAIN_ID)
        all_decoded_logs.append(decoded)

    import json
    # نمایش خروجی (با تبدیل HexBytes به رشته برای نمایش در JSON)
    class HexJsonEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, HexBytes):
                return obj.hex()
            return super().default(obj)

    print(json.dumps(all_decoded_logs, indent=2, cls=HexJsonEncoder))

except Exception as e:
    print(f"Critical Error: {e}")

Decoding logs for tx: 0xf952d08bafc9b2c7fa777a66f570d7809fa4f2758a5df3bca69deb5ae941844d
Transaction status: 1
Successfully fetched ABI from: 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14
[
  {
    "event_name": "Deposit",
    "args": {
      "dst": "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822",
      "wad": 1000000000000000
    },
    "address": "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"
  }
]


In [22]:
for i in txs_hash[-31:]:
  time.sleep(2)
  try:
    receipt = w3.eth.get_transaction_receipt(i)

    print(f"Transaction status: {receipt['status']}")

    all_decoded_logs = []
    for log in receipt['logs']:
        time.sleep(0.5)
        decoded = decode_log_fixed(w3, log, API_KEY, CHAIN_ID)
        all_decoded_logs.append(decoded)

    import json
    # نمایش خروجی (با تبدیل HexBytes به رشته برای نمایش در JSON)
    class HexJsonEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, HexBytes):
                return obj.hex()
            return super().default(obj)

    print(json.dumps(all_decoded_logs, indent=2, cls=HexJsonEncoder))

  except Exception as e:
    print(f"Critical Error: {e}")

Transaction status: 1
Successfully fetched ABI from: 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14
Successfully fetched ABI from: 0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1
Successfully fetched ABI from: 0x1238536071E1c677A632429e3655c799b22cDA52
Successfully fetched ABI from: 0x1238536071E1c677A632429e3655c799b22cDA52
[
  {
    "event_name": "Transfer",
    "args": {
      "src": "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822",
      "dst": "0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1",
      "wad": 3764500076879872
    },
    "address": "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14"
  },
  {
    "event_name": "Mint",
    "args": {
      "owner": "0x1238536071E1c677A632429e3655c799b22cDA52",
      "tickLower": 186440,
      "tickUpper": 186640,
      "sender": "0x1238536071E1c677A632429e3655c799b22cDA52",
      "amount": 33516997100150,
      "amount0": 0,
      "amount1": 3764500076879872
    },
    "address": "0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1"
  },
  {
    "event_name": "Transfer",